<a href="https://colab.research.google.com/github/JorgeDanilo/Data_Science/blob/master/span_probabilidade_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from collections import Counter, defaultdict
from machine_learning import split_data, precision, recall, f1_score
import math, random, re, glob


# Naive Bayes

Uma rede social não é muito boa se as pessoas não podem interagir. Assim, a *DataSciencester* tem um recurso popular que permite que os membros enviem mensagens para outros membros. E enquanto a maioria de seus membros são cidadãos responsáveis que enviam apenas mensagens educadas do tipo "como vai?", alguns canalhas insistentemente enviam spam a outros membros sobre esquemas de enriquecimento, produtos farmacêuticos sem receita médica e programas de credenciamento de ciência de dados para fins lucrativos. Seus usuários começaram a reclamar e, portanto, o diretor do departamento de mensagens solicitou que você usasse a ciência de dados para descobrir uma maneira de filtrar essas mensagens de spam.

## Um filtro de spam realmente bobo

Imagine um "universo" que consiste em receber uma mensagem escolhida aleatoriamente de todas as mensagens possíveis. Seja $S$ o evento "a mensagem é spam" e $V$ é o evento "a mensagem contém a palavra "viagra". Em seguida, o Teorema de Bayes nos diz que a probabilidade de a mensagem ser spam condicionada a conter a palavra "viagra" é:

$$P(S~|~V) = \frac{P(V~|~S)~P(S)}{P(V~|~S)~P(S) ~+~ P(V~|~\neg S)~P(\neg S)}$$

O numerador é a probabilidade de uma mensagem ser spam **e** conter viagra, enquanto o denominador é apenas a probabilidade de uma mensagem conter viagra. Portanto, você pode pensar nesse cálculo como simplesmente representando a proporção de mensagens do viagra que são spam.

Se temos uma grande coleção de mensagens que sabemos serem spam, e uma grande coleção de mensagens que sabemos não ser spam, podemos estimar facilmente $P(V~|~S)$ e $P(V~|~\neg S)$. Se, além disso, assumirmos que qualquer mensagem tem a mesma probabilidade de ser spam ou não spam (de modo que $P(S) = P(\neg S) = 0.5$), então:

$$P(S~|~V) = \frac{P(V~|~S)~P(S)}{P(V~|~S) ~+~ P(V~|~\neg S)}$$

Por exemplo, se 50% das mensagens de spam tiverem a palavra viagra, mas apenas 1% das mensagens não spam tiverem, a probabilidade de um determinado email contendo viagra ser spam é:

$$P(S~|~V) = \frac{0.5}{0.5 + 0.01} = 98\%$$ 

## Implementação

Agora temos todas as peças que precisamos para construir nosso classificador. Primeiro, vamos criar uma função simples para transformar as mensagens em palavras distintas. Primeiro, converteremos cada mensagem em minúscula. Para isso, use `re.findall()` para extrair "palavras" consistindo de letras, números e apóstrofos; e finalmente use `set()` para obter apenas as palavras distintas:

In [2]:
def tokenize(message):
  messsage = message.lower()                        # convert to lowercase
  all_words = re.findall("[a-z0-9']+", message)     # extract the words
  return set(all_words)                             # remove duplicates

Nossa segunda função contará as palavras de um conjunto de treinamento de mensagens rotuladas. Teremos que retornar um dicionário cujas chaves sejam palavras e cujo valores sejam listas de dois elementos [spam_count, non_spam_count] correspondendo a quantas vezes vimos essas palavras em mensagens de spam e não-spam.

In [10]:
def count_words(training_set):
  """training set consists of pairs (message, is_spam)"""
  counts = defaultdict(lambda: [0, 0])
  for message, is_spam in training_set:
    for word in tokenize(message):
      counts[word][0 if is_spam else 1] += 1
  return counts

Nosso próximo passo é transformar essas contagens em probabilidades estimadas usando a suavização descrita anteriormente. Nossa função retornará uma lista de três valores: a palavra, a probabilidade de ver essa palavra em uma mensagem de spam e a probabilidade de ver essa palavra em uma mensagem não-spam:

In [6]:
def word_probabilities(counts, total_spams, total_non_spams, k=0.5):
  """turn the word_counts int a list of triplets
  w, p(w | spam) and p(w | ~ spam) """
  return [(w,
          (spam + k) / (total_spams + 2 * k),
          (non_spam + k) / (total_non_spams + 2 * k))
          for w, (spam, non_spam) in counts.items()]

A última parte é usar essas probabilidades de palavras (e nossas suposições Naive Bayes) para atribuir probabilidades às mensagens:

In [52]:
def spam_probability(word_probs, message):
  message_words = tokenize(message)
  log_prob_if_spam = log_prob_if_not_spam = 0.0

  for word, prob_if_spam, prob_if_not_spam in word_probs:

    # for each word in the message.
    # add the log probability of seeing it
    if word in message_words:
      log_prob_if_spam += math.log(prob_if_spam)
      log_prob_if_not_spam += math.log(prob_if_not_spam)

    # for each word that's not in the message
    # add the log probability of _not_ seeing it
    else:
      log_prob_if_spam += math.log(1.0 - prob_if_spam)
      log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)

  prob_if_spam = math.exp(log_prob_if_spam)
  prob_if_not_spam = math.exp(log_prob_if_not_spam)
  return prob_if_spam / (prob_if_spam + prob_if_not_spam)

Podemos colocar tudo isso junto em nosso classificador Naive Bayes:

In [47]:
class NaiveBayesClassifier:

  def __init__(self, k=0.5):
    self.k = k
    self.word_probs = []

  def train(self, training_set):

    # count spam and non-spam messages
    num_spams = len([is_spam
                     for message, is_spam in training_set
                     if is_spam])
    num_non_spams = len(training_set) - num_spams
    
    # run training data through our "pipeline"
    word_counts = count_words(training_set)
    self.word_probs = word_probabilities(word_counts,
                                         num_spams,
                                         num_non_spams,
                                         self.k)
    
  def classify(self, message):
    return spam_probability(self.word_probs, message)

In [66]:
def get_subject_data(path):

  data = []

  subject_regex = re.compile(r"^Subject:\s+")

  for fn in glob.glob(path):
    is_spam = "ham" not in fn

    with open(fn, 'r', encoding='ISO-8859-1') as file:
      for line in file:
        if line.startswith("Subject:"):
          subject = subject_regex.sub("", line).strip()
          data.append((subject, is_spam))

  return data


In [67]:
path = "/content/data/spam/*/*"
data = get_subject_data(path)
print(len(data), "messagens lidas.")

898 messagens lidas.


Agora a gente pode dividir os dados em conjunto de treino e conjunto de teste, e depois estaremos prontos para executar o classificador

In [68]:
random.seed(0)
train_data, test_data = split_data(data, 0.75)
classifier = NaiveBayesClassifier()
classifier.train(train_data)

Agora vamos ver como nosso modelo se saiu:

In [69]:
# tripplets (subject, actual is_spam, predicted spam probability)
classified = [(subject, is_spam, classifier.classify(subject))
              for subject, is_spam in test_data]

# assume that spam_probability > 0.5 correponds to spam prediction
# and count the combinations of (actual is_spam, predicted is_spam)
counts = Counter((is_spam, spam_probability > 0.5)
                  for _, is_spam, spam_probability in classified)

print(counts)

Counter({(True, True): 111, (False, False): 52, (False, True): 46, (True, False): 6})


In [70]:
tp = counts[(True, True)]
fp = counts[(False, True)]
tn = counts[(False, False)]
fn = counts[(True, False)]

A nossa precião foi em torno de 71%

In [71]:
print("Precisão: ", precision(tp, fp, fn, tn))

Precisão:  0.7070063694267515


Podemos ver as palavras que mais tem spam

In [73]:
def p_spam_given_word(word_prob):
  word, prob_if_spam, prob_if_not_spam = word_prob
  return prob_if_spam / (prob_if_spam + prob_if_not_spam)

In [74]:
words = sorted(classifier.word_probs, key=p_spam_given_word)

spammiest_words = words[-5:]
hammiest_words = words[:5]

In [75]:
print("spammiest_words", spammiest_words)

spammiest_words [('6', 0.02454780361757106, 0.0016778523489932886), ('zzzz', 0.027131782945736434, 0.0016778523489932886), ('ocial', 0.029715762273901807, 0.0016778523489932886), ('5', 0.03488372093023256, 0.0016778523489932886), ('ow', 0.03488372093023256, 0.0016778523489932886)]


In [77]:
print("hammiest_words", hammiest_words)

hammiest_words [('zzzzteana', 0.0012919896640826874, 0.025167785234899327), ('be', 0.0012919896640826874, 0.015100671140939598), ('ed', 0.0012919896640826874, 0.015100671140939598), ('crash', 0.0012919896640826874, 0.015100671140939598), ('deal', 0.0012919896640826874, 0.015100671140939598)]


As palavras mais relacionadas com spam são "money", "systemworks", "rates", "sale" e "year", todas relacionadas à tentativa de levar as pessoas a comprar coisas. E as palavras mais relacionadas a mensagens não spam são "spambayes", "users", "razor", "zzzzteana" e "sadev", a maioria das quais parece relacionada à prevenção de spam, curiosamente.

### Como poderíamos obter um melhor desempenho?

Uma maneira óbvia seria obter mais dados para treinar. Há várias maneiras de melhorar o modelo também. Aqui estão algumas possibilidades que você pode tentar:

* Observe o conteúdo da mensagem, não apenas a linha de assunto. Você precisa ter cuidado ao lidar com os cabeçalhos das mensagens.

* Nosso classificador leva em conta todas as palavras que aparecem no conjunto de treinamento, mesmo as que aparecem apenas uma vez. Modifique o classificador para aceitar um limiar `min_count` opcional e ignorar os tokens que não aparecem pelo menos essa quantidade de vezes.

* O tokenizador não tem noção de palavras semelhantes (por exemplo, "cheap" e "cheapest"). Modifique o classificador para obter uma função opcional `stemmer` que converta palavras em *classes de equivalência*  de palavras. Por exemplo, uma função `stemmer` realmente simples pode ser: